<a href="https://colab.research.google.com/github/kimdesok/Recommender_Systems/blob/main/Recommender_Movies_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%cd /content
!pip install kaggle --upgrade
!kaggle datasets download -d rounakbanik/the-movies-dataset

/content
 93% 211M/228M [00:01<00:00, 234MB/s]
100% 228M/228M [00:01<00:00, 208MB/s]


In [2]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.4 MB/s eta 0:00:00


In [6]:
!unzip /content/the-movies-dataset.zip -d /content/the-movies-dataset

Archive:  /content/the-movies-dataset.zip
  inflating: /content/the-movies-dataset/credits.csv  
  inflating: /content/the-movies-dataset/keywords.csv  
  inflating: /content/the-movies-dataset/links.csv  
  inflating: /content/the-movies-dataset/links_small.csv  
  inflating: /content/the-movies-dataset/movies_metadata.csv  
  inflating: /content/the-movies-dataset/ratings.csv  
  inflating: /content/the-movies-dataset/ratings_small.csv  


In [7]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [55]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('./the-movies-dataset/movies_metadata.csv', low_memory=False)[:20000]

# Print the first three rows
metadata.head(3)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0


In [56]:
# Calculate mean of vote average column
C = metadata['vote_average'].mean()
print(C)


6.004100410041004


In [10]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

346.0


In [11]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(2004, 24)

In [12]:
metadata.shape

(20000, 24)

In [13]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

## Recommendation by the weighted average ratings

In [15]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)


,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.400783
834,The Godfather,6024.0,8.5,8.364430
12481,The Dark Knight,12269.0,8.3,8.237029
2843,Fight Club,9678.0,8.3,8.220752
292,Pulp Fiction,8670.0,8.3,8.211892
522,Schindler's List,4436.0,8.3,8.133881
5481,Spirited Away,3968.0,8.3,8.115860
351,Forrest Gump,8147.0,8.2,8.110540
2211,Life Is Beautiful,3643.0,8.3,8.100857
1178,The Godfather: Part II,3418.0,8.3,8.088953


## Content based Recommender

In [16]:
#Recommending movies similar to a particular movie.

pd.set_option('display.max_colwidth', None)  #print all sentences with no shortening

#Print plot overviews of the first 5 movies.
metadata['overview'].head()


0                                                                                                Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.
1    When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.
2                                                                        A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry

In [17]:
metadata['original_title'].head()

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: original_title, dtype: object

In [18]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(20000, 47487)

In [19]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names_out()[5000:5010]

array(['blow', 'blower', 'blowgun', 'blowhard', 'blowing', 'blown',
       'blowout', 'blows', 'blu', 'bludgeoned'], dtype=object)

In [20]:
#Cosine similarity

# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [21]:
cosine_sim.shape


(20000, 20000)

In [22]:
cosine_sim[1]


array([0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
       0.        ])

In [23]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()


In [24]:
indices[:10]


title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [25]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]


In [50]:
get_recommendations('The Dark Knight Rises')

4780                                   The Fluffer
5844                                     My Girl 2
6906                                   The Witches
15829                                    Beach Red
7903                                     All of Me
18355                                   Picture Me
3013                           Ride with the Devil
6940                   The Hunchback of Notre Dame
12583    Harold & Kumar Escape from Guantanamo Bay
407                           The Age of Innocence
Name: title, dtype: object

In [51]:
get_recommendations('The Godfather')

3844                Lost Souls
3244     The Perils of Pauline
183                Nine Months
1379                     Evita
1649     Stranger in the House
1415                     Touch
12194              Elite Squad
12626                 Iron Man
10093           Samaritan Girl
3415                    Taffin
Name: title, dtype: object

In [53]:
# Load keywords and credits
credits = pd.read_csv('././the-movies-dataset/credits.csv')[:20000]
print(credits.columns)
print(credits[:3])
keywords = pd.read_csv('././the-movies-dataset/keywords.csv')[:20000]
print(keywords.columns)
print(keywords[:3])

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')


Index(['cast', 'crew', 'id'], dtype='object')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [54]:
# Print the first two movies of your newly merged metadata
metadata.head(2)

index  adult  \
0      0  False   
1      1  False   

                                                                                                                                   belongs_to_collection  \
0  {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}   
1                                                                                                                                                    NaN   

     budget                        genres  \
0  30000000   [animation, comedy, family]   
1  65000000  [adventure, fantasy, family]   

                               homepage    id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story   862  tt0114709                en   
1                                   NaN  8844  tt0113497                en   

  original_title  ...                                    tagline      title  \
0      Toy Story  ...                                        NaN  Toy Story   
1        Jumanji  ...  Roll the dice and unleash the excitement!    Jumanji   

   video vote_average vote_count                                         cast  \
0  False          7.7     5415.0             [tomhanks, timallen, donrickles]   
1  False          6.9     2413.0  [robinwilliams, jonathanhyde, kirstendunst]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [28]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)


In [29]:
# Import Numpy
import numpy as np


In [30]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [31]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


In [32]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)


In [33]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)


,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children's book]","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [34]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [35]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)


In [36]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


In [37]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)


In [38]:
metadata[['soup']].head(2)


,soup
0,jealousy toy boy tomhanks timallen donrickles johnlasseter animation comedy family
1,boardgame disappearance basedonchildren'sbook robinwilliams jonathanhyde kirstendunst joejohnston adventure fantasy family


## Recommender using CountVectorizer

In [39]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])


In [40]:
count_matrix.shape


(20059, 36686)

In [41]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)


In [42]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])


In [43]:
get_recommendations('The Dark Knight Rises', cosine_sim2)


12519      The Dark Knight
10147        Batman Begins
9255                Shiner
9811       Amongst Friends
7716              Mitchell
516      Romeo Is Bleeding
11393         The Prestige
10790       Helter Skelter
18810            Last Exit
4492       An Innocent Man
Name: title, dtype: object

In [44]:
get_recommendations('The Godfather', cosine_sim2)


1927            The Godfather: Part III
1192             The Godfather: Part II
15500                   The Rain People
18810                         Last Exit
7945     The Night of the Following Day
18131                 The Son of No One
7716                           Mitchell
1179                     Apocalypse Now
1641                   Ill Gotten Gains
3473         Jails, Hospitals & Hip-Hop
Name: title, dtype: object

In [45]:
get_recommendations('Taxi Driver', cosine_sim2)


7609                      Mean Streets
15                              Casino
1184                        GoodFellas
4960                   Falling in Love
423                       A Bronx Tale
18810                        Last Exit
17730                    Street Scenes
7716                          Mitchell
1199                       Raging Bull
1926     The Last Temptation of Christ
Name: title, dtype: object

## Collaborative Filtering
>* User-based filtering - Recommending based on the similar user's preferrence.  First users with similar interests should be identified.
>* Item-based filtering - Recommending based on the items that the user's given the same ratings. Alice, Bob, and Eve have given 5 stars to the item A and the item B.  Then the items A and B are similar. Thus, if someone buys the item A, the system recommend the item B.

Data available at https://grouplens.org/datasets/movielens/